In [ ]:
!pip install --upgrade transformers torch accelerate

In [3]:
!pip uninstall pyarrow -y
!pip install -q pyarrow==12.0.0

Found existing installation: pyarrow 10.0.1
Uninstalling pyarrow-10.0.1:
  Successfully uninstalled pyarrow-10.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 19.7 MB/s eta 0:00:00


In [4]:
import pyarrow
print(pyarrow.__version__)

12.0.0


In [5]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00


In [2]:
import accelerate
print(accelerate.__version__)

0.27.2


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
ds = 'manual' # 'mrpc','manual'
# sentiment = 'positive' # 'positive', 'negative'

folder_input_path = '/content/drive/My Drive/Colab Notebooks/5_Corpora/corpora/'
folder_pretrained_path = '/content/drive/My Drive/Colab Notebooks/8_Text_Paraphrasing/pretrained/pegasus'
csv_file_path = f'{ds}-triplet-corpus.csv'

In [8]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset
import torch
import pandas as pd

In [9]:
def preprocess_data(examples, tokenizer):
    inputs = ["paraphrase: " + doc for doc in examples["input_text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    labels = tokenizer(examples["target_text"], max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
def fine_tune_model(dataset_path, output_dir):
    # Load the tokenizer and model specifically tailored for paraphrasing
    model_name = 'tuner007/pegasus_paraphrase'
    tokenizer = PegasusTokenizer.from_pretrained(model_name)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

    # Load the dataset
    df = pd.read_csv(dataset_path)
    dataset = df[['original', 'positive']].rename(columns={'original': 'input_text', 'positive': 'target_text'}) # Adjust based on sentiment
    dataset = Dataset.from_pandas(dataset)

    # Preprocess the dataset
    tokenized_dataset = dataset.map(lambda x: preprocess_data(x, tokenizer), batched=True)

    # Training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        predict_with_generate=True,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=3,
        weight_decay=0.01,
    )

    # Initialize the trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        eval_dataset=tokenized_dataset, # Assuming you want to use the same dataset for simplicity; adjust as needed
        tokenizer=tokenizer,
    )

    # Train the model
    trainer.train()

    # Save the fine-tuned model and tokenizer
    # model.save_pretrained(output_dir)
    # tokenizer.save_pretrained(output_dir)

    return model, tokenizer

In [11]:
# Given that `prithivida/parrot_paraphraser_on_T5` is already fine-tuned for paraphrasing, you may not need to fine-tune further for similar tasks
# However, if you have a specific dataset or wish to further tailor the model, you can proceed with training as demonstrated earlier

model, tokenizer = fine_tune_model(folder_input_path + csv_file_path, folder_pretrained_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/212 [00:00<?, ? examples/s]

IndexError: index out of range in self

In [ ]:
# Reminder: This example is structured for demonstrating how to load and potentially fine-tune a specific paraphrasing model
# For actual paraphrasing (inference), use the model's `generate` function with appropriate parameters

In [ ]:
# Ensure model is in evaluation mode
model.eval()

# Move model to the appropriate device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare your input text
input_text = "This is the sentence I want to paraphrase."
input_text = "paraphrase: " + input_text  # Add any necessary prefix
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# Generate the paraphrase
outputs = model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)

# Decode generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Generated text: {generated_text}")


Generated text: 
